In [1]:
import classes_taskonomy as classes

# modified visualpriors library
from transforms import VisualPriorRepresentation

import os
import pandas as pd

import torch.utils.model_zoo # required to load nets
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

In [2]:
#VisualPrior.viable_feature_tasks
TASKS = ('autoencoding','depth_euclidean','jigsaw','reshading',
         'colorization', 'edge_occlusion','keypoints2d','room_layout',
         'curvature','edge_texture','keypoints3d','segment_unsup2d',
         'class_object','egomotion','nonfixated_pose','segment_unsup25d',
         'class_scene','fixated_pose','normal','segment_semantic',
         'denoising','inpainting','point_matching','vanishing_point')


In [3]:
DATASET_NAMES = ('places1', 'places2', 'oasis')
SCALE_NAMES = ('scale2','scale4','scale8','scale16','scale32')

In [4]:
DATA_PATH = './data_256x256'
BEHAVIOR_PATH = './behavior'
RESULTS_PATH = './results_taskonomy'

In [6]:
#VisualPrior.viable_feature_tasks
MODEL_NAMES = ('autoencoding','depth_euclidean','jigsaw','reshading',
               'colorization', 'edge_occlusion','keypoints2d','room_layout',
               'curvature','edge_texture','keypoints3d','segment_unsup2d',
               'class_object','egomotion','nonfixated_pose','segment_unsup25d',
               'class_scene','fixated_pose','normal','segment_semantic',
               'denoising','inpainting','point_matching','vanishing_point')


In [7]:
#IMAGE_TRANSFORMS = ('untranformed')

In [10]:
for model_name in MODEL_NAMES:
    print(model_name)

    # Import taskonomy model...
    VisualPriorRepresentation._load_unloaded_nets([model_name])
    net = VisualPriorRepresentation.feature_task_to_net[model_name]
    
    # ...and create activation extractor from it
    _, eval_nodes = get_graph_node_names(net)
    return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}
    activation_extractor = create_feature_extractor(net, return_nodes=return_nodes)


    for dataset_name in DATASET_NAMES:
        print(dataset_name)
        for scale_name in SCALE_NAMES:
            
            dataset = classes.ImageDataset(
                os.path.join(DATA_PATH, dataset_name, 'untransformed', scale_name))
                        
            correlations = classes.calculate_dataset_correlation(dataset, activation_extractor)
            correlations.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'correlations.csv'), index=False, header=False)


            selfsimilarity = classes.calculate_dataset_self_similarity(dataset, activation_extractor)
            selfsimilarity.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'selfsimilarity.csv'), index=False, header=False)

            l2norm = classes.calculate_dataset_l2norm(dataset, activation_extractor)
            l2norm.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'l2norm.csv'), index=False, header=False)
        
            break

autoencoding
places1


/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/max/minico

places1


/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/max/minico

places2


FileNotFoundError: [Errno 2] No such file or directory: './data_256x256/places2/untransformed/scale2/full'

# Linear modelling

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut


X = StandardScaler().fit_transform(result)
y = StandardScaler().fit_transform(pd.DataFrame(places_images.beauty_ratings))
X.shape, y.shape
loo, predictions = LeaveOneOut(), []

for train_idx, predict_idx in loo.split(X):
    glm = LinearRegression().fit(X[train_idx], y[train_idx])
    predictions.append(glm.predict(X[predict_idx]).item())

predictions = pd.Series(predictions)

pearsonr(predictions, places_images.beauty_ratings)